In [1]:
from IPython.display import Markdown, display
bash = lambda commands: display(Markdown("```bash\n" + ' && \n'.join(commands) + "\n```"))

In [19]:
import re

def summary(model_list, profile_path="profile.txt"):
    # Read the profile file
    with open(profile_path, 'r') as file:
        content = file.read()

    # Prepare regex patterns
    model_blocks = re.split(r"\n(?=Model:)", content)

    # Prepare summary dictionary
    summary = {}

    for block in model_blocks:
        model_match = re.search(r"Model:\s*(\S+)", block)
        total_match = re.search(r"Total parameters:\s*([\d,]+)", block)
        nonzero_match = re.search(r"Nonzero parameters:\s*([\d,]+)", block)
        sparsity_match = re.search(r"Sparsity:\s*([\d\.]+)%", block)

        if model_match:
            model = model_match.group(1)
            if model in model_list:
                total_params = int(total_match.group(1).replace(",", ""))
                nonzero_params = int(nonzero_match.group(1).replace(",", ""))
                pruned_params = total_params - nonzero_params
                sparsity = float(sparsity_match.group(1))

                summary[model] = {
                    "Total": total_params,
                    "Nonzero": nonzero_params,
                    "Pruned": pruned_params,
                    "Sparsity": sparsity
                }

    # Generate Markdown table
    md_table = "| Model | Total Parameters | Nonzero Parameters | Pruned Parameters | Sparsity (%) |\n"
    md_table += "|-------|------------------|--------------------|-------------------|--------------|\n"

    for model in model_list:
        if model in summary:
            data = summary[model]
            md_table += (
                f"| {model} | {data['Total']:,} | {data['Nonzero']:,} | {data['Pruned']:,} | {data['Sparsity']:.2f} |\n"
            )
        else:
            print(f"Warning: {model} not found in profile file.")

    display(Markdown(md_table))


# One-shot unmasked:

In [17]:
sparsities = [f"sparse_{i}{r}" for i in [a/10 for a in range(10)] for r in ["", "_retrained"]]

bash([
  "cd archive/one_shot",
f"""for i in {' '.join([str(s) for s in sparsities])}; do
  echo ${{i}} &&
  python3 ../../profile.py ${{i}} >> profile.txt &&
  echo "" >> profile.txt
done""",
  "cd ../.."
])

```bash
cd archive/one_shot && 
for i in sparse_0.0 sparse_0.0_retrained sparse_0.1 sparse_0.1_retrained sparse_0.2 sparse_0.2_retrained sparse_0.3 sparse_0.3_retrained sparse_0.4 sparse_0.4_retrained sparse_0.5 sparse_0.5_retrained sparse_0.6 sparse_0.6_retrained sparse_0.7 sparse_0.7_retrained sparse_0.8 sparse_0.8_retrained sparse_0.9 sparse_0.9_retrained; do
  echo ${i} &&
  python3 ../../profile.py ${i} >> profile.txt &&
  echo "" >> profile.txt
done && 
cd ../..
```

In [21]:
sparsities = [f"sparse_{i}" for i in [a/10 for a in range(10)]]
summary(sparsities, "archive/one_shot/profile.txt")

| Model | Total Parameters | Nonzero Parameters | Pruned Parameters | Sparsity (%) |
|-------|------------------|--------------------|-------------------|--------------|
| sparse_0.0 | 78,719,936 | 78,719,890 | 46 | 0.00 |
| sparse_0.1 | 78,719,936 | 70,908,043 | 7,811,893 | 9.92 |
| sparse_0.2 | 78,719,936 | 63,096,152 | 15,623,784 | 19.85 |
| sparse_0.3 | 78,719,936 | 55,284,260 | 23,435,676 | 29.77 |
| sparse_0.4 | 78,719,936 | 47,472,369 | 31,247,567 | 39.69 |
| sparse_0.5 | 78,719,936 | 39,660,478 | 39,059,458 | 49.62 |
| sparse_0.6 | 78,719,936 | 31,848,587 | 46,871,349 | 59.54 |
| sparse_0.7 | 78,719,936 | 24,036,696 | 54,683,240 | 69.47 |
| sparse_0.8 | 78,719,936 | 16,224,804 | 62,495,132 | 79.39 |
| sparse_0.9 | 78,719,936 | 8,412,913 | 70,307,023 | 89.31 |


In [23]:
sparsities = [f"sparse_{i}_retrained" for i in [a/10 for a in range(10)]]
summary(sparsities, "archive/one_shot/profile.txt")

| Model | Total Parameters | Nonzero Parameters | Pruned Parameters | Sparsity (%) |
|-------|------------------|--------------------|-------------------|--------------|
| sparse_0.0_retrained | 78,719,936 | 78,719,909 | 27 | 0.00 |
| sparse_0.1_retrained | 78,719,936 | 78,719,245 | 691 | 0.00 |
| sparse_0.2_retrained | 78,719,936 | 78,695,713 | 24,223 | 0.03 |
| sparse_0.3_retrained | 78,719,936 | 78,718,790 | 1,146 | 0.00 |
| sparse_0.4_retrained | 78,719,936 | 78,719,595 | 341 | 0.00 |
| sparse_0.5_retrained | 78,719,936 | 78,718,822 | 1,114 | 0.00 |
| sparse_0.6_retrained | 78,719,936 | 78,716,798 | 3,138 | 0.00 |
| sparse_0.7_retrained | 78,719,936 | 78,688,093 | 31,843 | 0.04 |
| sparse_0.8_retrained | 78,719,936 | 78,484,113 | 235,823 | 0.30 |
| sparse_0.9_retrained | 78,719,936 | 77,994,874 | 725,062 | 0.92 |


# One-shot masked:

In [ ]:
sparsities = [f"sparse_{i}{r}" for i in sorted(set([a/10 for a in range(10)] + [1-0.5**a for a in range(1,5)] + [1-0.75**a for a in range(1,9)])) for r in ["", "_retrained"]]

bash([
  "cd one_shot",
f"""for i in {' '.join([str(s) for s in sparsities])}; do
  echo ${{i}} &&
  python3 ../profile.py ${{i}} >> profile.txt &&
  echo "" >> profile.txt
done""",
  "cd .."
])

```bash
cd one_shot && 
for i in sparse_0.0 sparse_0.0_retrained sparse_0.1 sparse_0.1_retrained sparse_0.2 sparse_0.2_retrained sparse_0.25 sparse_0.25_retrained sparse_0.3 sparse_0.3_retrained sparse_0.4 sparse_0.4_retrained sparse_0.4375 sparse_0.4375_retrained sparse_0.5 sparse_0.5_retrained sparse_0.578125 sparse_0.578125_retrained sparse_0.6 sparse_0.6_retrained sparse_0.68359375 sparse_0.68359375_retrained sparse_0.7 sparse_0.7_retrained sparse_0.75 sparse_0.75_retrained sparse_0.7626953125 sparse_0.7626953125_retrained sparse_0.8 sparse_0.8_retrained sparse_0.822021484375 sparse_0.822021484375_retrained sparse_0.86651611328125 sparse_0.86651611328125_retrained sparse_0.875 sparse_0.875_retrained sparse_0.8998870849609375 sparse_0.8998870849609375_retrained sparse_0.9 sparse_0.9_retrained sparse_0.9375 sparse_0.9375_retrained; do
  echo ${i} &&
  python3 ../profile.py ${i} >> profile.txt &&
  echo "" >> profile.txt
done
```

In [24]:
sparsities = [f"sparse_{i}_retrained" for i in [a/10 for a in range(10)]]
summary(sparsities, "one_shot/profile.txt")

| Model | Total Parameters | Nonzero Parameters | Pruned Parameters | Sparsity (%) |
|-------|------------------|--------------------|-------------------|--------------|
| sparse_0.0_retrained | 78,719,936 | 78,719,904 | 32 | 0.00 |
| sparse_0.1_retrained | 78,719,936 | 70,908,045 | 7,811,891 | 9.92 |
| sparse_0.2_retrained | 78,719,936 | 63,096,154 | 15,623,782 | 19.85 |
| sparse_0.3_retrained | 78,719,936 | 55,284,262 | 23,435,674 | 29.77 |
| sparse_0.4_retrained | 78,719,936 | 47,472,371 | 31,247,565 | 39.69 |
| sparse_0.5_retrained | 78,719,936 | 39,660,480 | 39,059,456 | 49.62 |
| sparse_0.6_retrained | 78,719,936 | 31,848,588 | 46,871,348 | 59.54 |
| sparse_0.7_retrained | 78,719,936 | 24,036,698 | 54,683,238 | 69.47 |
| sparse_0.8_retrained | 78,719,936 | 16,224,806 | 62,495,130 | 79.39 |
| sparse_0.9_retrained | 78,719,936 | 8,412,915 | 70,307,021 | 89.31 |


# Iterative masked:

In [12]:
sparsities = [f"iter_{i}{r}" for i in range(1,9) for r in ["", "_retrained"]]

bash([
  "cd iterative_0.25",
f"""for i in {' '.join([str(s) for s in sparsities])}; do
  echo ${{i}} &&
  python3 ../profile.py ${{i}} >> profile.txt &&
  echo "" >> profile.txt
done""",
  "cd .."
])

```bash
cd iterative_0.25 && 
for i in iter_1 iter_1_retrained iter_2 iter_2_retrained iter_3 iter_3_retrained iter_4 iter_4_retrained iter_5 iter_5_retrained iter_6 iter_6_retrained iter_7 iter_7_retrained iter_8 iter_8_retrained; do
  echo ${i} &&
  python3 ../profile.py ${i} >> profile.txt &&
  echo "" >> profile.txt
done && 
cd ..
```

In [27]:
sparsities = [f"iter_{i}_retrained" for i in range(1,9)]
summary(sparsities, "iterative_0.25/profile.txt")

| Model | Total Parameters | Nonzero Parameters | Pruned Parameters | Sparsity (%) |
|-------|------------------|--------------------|-------------------|--------------|
| iter_1_retrained | 78,719,936 | 59,190,208 | 19,529,728 | 24.81 |
| iter_2_retrained | 78,719,936 | 44,542,911 | 34,177,025 | 43.42 |
| iter_3_retrained | 78,719,936 | 33,557,440 | 45,162,496 | 57.37 |
| iter_4_retrained | 78,719,936 | 25,318,334 | 53,401,602 | 67.84 |
| iter_5_retrained | 78,719,936 | 19,139,008 | 59,580,928 | 75.69 |
| iter_6_retrained | 78,719,936 | 14,504,512 | 64,215,424 | 81.57 |
| iter_7_retrained | 78,719,936 | 11,028,639 | 67,691,297 | 85.99 |
| iter_8_retrained | 78,719,936 | 8,421,735 | 70,298,201 | 89.30 |


In [29]:
sparsities = [f"sparse_{s}_retrained" for s in [1-0.75**a for a in range(1,9)]]
summary(sparsities, "one_shot/profile.txt")

| Model | Total Parameters | Nonzero Parameters | Pruned Parameters | Sparsity (%) |
|-------|------------------|--------------------|-------------------|--------------|
| sparse_0.25_retrained | 78,719,936 | 59,190,208 | 19,529,728 | 24.81 |
| sparse_0.4375_retrained | 78,719,936 | 44,542,911 | 34,177,025 | 43.42 |
| sparse_0.578125_retrained | 78,719,936 | 33,557,439 | 45,162,497 | 57.37 |
| sparse_0.68359375_retrained | 78,719,936 | 25,318,336 | 53,401,600 | 67.84 |
| sparse_0.7626953125_retrained | 78,719,936 | 19,139,008 | 59,580,928 | 75.69 |
| sparse_0.822021484375_retrained | 78,719,936 | 14,504,511 | 64,215,425 | 81.57 |
| sparse_0.86651611328125_retrained | 78,719,936 | 11,028,639 | 67,691,297 | 85.99 |
| sparse_0.8998870849609375_retrained | 78,719,936 | 8,421,734 | 70,298,202 | 89.30 |
